<a href="https://colab.research.google.com/github/mahesh-from-sirsi/All_My_AI_Work/blob/main/BuildFastWithAi_2_5_Contextual_Compressors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Contextual Compressors

Langchain Documentation: https://python.langchain.com/docs/how_to/contextual_compression/

In [ ]:
%pip install -qU langchain langchain-community langchain-openai langchain-text-splitters faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

documents = WebBaseLoader("https://en.wikipedia.org/wiki/Big_Mac_Index").load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever(search_kwargs={"k": 5})


In [ ]:
docs = retriever.invoke("What is this index?")
docs

[Document(id='fbe94ab4-27ed-44b2-8ad9-e5069c5c767b', metadata={'source': 'https://en.wikipedia.org/wiki/Big_Mac_Index', 'title': 'Big Mac Index - Wikipedia', 'language': 'en'}, page_content='^ "What Is the Big Mac index?". Investopedia. Archived from the original on 29 April 2022. Retrieved 29 April 2022.'),
 Document(id='e792009f-de1d-410e-82dc-2255ae45264d', metadata={'source': 'https://en.wikipedia.org/wiki/Big_Mac_Index', 'title': 'Big Mac Index - Wikipedia', 'language': 'en'}, page_content='The Big Mac Index index page\xa0– contains Big Mac Index data dating back to 1997 (Economist.com subscription required for detail)\nBig Mac versus iTunes Aplia\nBig Mac Index in text format for analysis\nAuthority control databases GND'),
 Document(id='7c3f4252-ad31-458d-b56f-065fc4f4501d', metadata={'source': 'https://en.wikipedia.org/wiki/Big_Mac_Index', 'title': 'Big Mac Index - Wikipedia', 'language': 'en'}, page_content='Privacy policy\nAbout Wikipedia\nDisclaimers\nContact Wikipedia\nCode

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors.chain_extract import LLMChainExtractor
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What is this index?"
)
compressed_docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Big_Mac_Index', 'title': 'Big Mac Index - Wikipedia', 'language': 'en'}, page_content='"What Is the Big Mac index?"'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Big_Mac_Index', 'title': 'Big Mac Index - Wikipedia', 'language': 'en'}, page_content='The Big Mac Index index page'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Big_Mac_Index', 'title': 'Big Mac Index - Wikipedia', 'language': 'en'}, page_content='Big Mac Index'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Big_Mac_Index', 'title': 'Big Mac Index - Wikipedia', 'language': 'en'}, page_content='Big Mac Index - Wikipedia'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Big_Mac_Index', 'title': 'Big Mac Index - Wikipedia', 'language': 'en'}, page_content='"Big Mac Index"')]